In [2]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import torchvision
from torchvision.datasets import MNIST
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.nn as nn 

In [5]:
a=[0,1,2,3,4,5]
b=[34,23,34,45,56,45]

print(f'\n zip converts list to tuple ')
for x in zip(a, b):
    print(x,type(x))

print(f'\n zip converts list to int by one by')
for x,y in zip(a, b):
    print(x,type(x))

print(f'\n zip slice and converts list to tuple')
for x in zip(a,a[1:3]):
    print(x,type(x))

print(f'\n zip slice and converts list to int')
for x,y in zip(a,a[1:3]):
    print(x,type(x))


 zip converts list to tuple 
(0, 34) <class 'tuple'>
(1, 23) <class 'tuple'>
(2, 34) <class 'tuple'>
(3, 45) <class 'tuple'>
(4, 56) <class 'tuple'>
(5, 45) <class 'tuple'>
(5, 45)

 zip converts list to int by one by
0 <class 'int'>
1 <class 'int'>
2 <class 'int'>
3 <class 'int'>
4 <class 'int'>
5 <class 'int'>

 zip slice and converts list to tuple
(0, 1) <class 'tuple'>
(1, 2) <class 'tuple'>

 zip slice and converts list to int
0 <class 'int'>
1 <class 'int'>


In [24]:
sizes=[1,16,32,32,64,128,64,32,32,1]

for 
layers = nn.ModuleList([nn.Linear(in_f, out_f) for in_f, out_f in zip(sizes, sizes[1:])])
